In [328]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import time
import pandas as pd

PATH = "/Users/FaiqRasulov/Desktop/chromedriver"
driver = webdriver.Chrome(PATH)

PATH = "/Users/FaiqRasulov/Desktop/chromedriver"
driver = webdriver.Chrome(PATH)

urls = [
    "https://bazarstore.az/collections/pendir?filter.p.m.custom.lokasyonlar=4101",
    "https://bazarstore.az/collections/sud-məhsullari?filter.p.m.custom.lokasyonlar=4101",
    "https://bazarstore.az/collections/qatiq?filter.p.m.custom.lokasyonlar=4101",
    "https://bazarstore.az/collections/səhər-yeməyi?filter.p.m.custom.lokasyonlar=4101",
    "https://bazarstore.az/collections/sud?filter.p.m.custom.lokasyonlar=4101",
    "https://bazarstore.az/collections/kərə-yagi?filter.p.m.custom.lokasyonlar=4101",
    "https://bazarstore.az/collections/marqarin?filter.p.m.custom.lokasyonlar=4101"
]

def handle_popup():
    try:
        exit_button = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//div[@class='css-11f6z0a']")))
        actions = ActionChains(driver)
        actions.move_to_element(exit_button).click().perform()
        time.sleep(1)  
    except:
        pass


def slow_scroll_down(speed, max_attempts=3):
    current_scroll_position = 0
    attempts = 0
    while True:
        prev_scroll_position = current_scroll_position
        current_scroll_position += speed
        driver.execute_script(f"window.scrollTo(0, {current_scroll_position});")
        time.sleep(1)  
        if current_scroll_position >= driver.execute_script("return document.body.scrollHeight"):
            break
        if current_scroll_position == prev_scroll_position:
            attempts += 1
            if attempts >= max_attempts:
                break
        else:
            attempts = 0

def get_kg_price():
    try:
        kg_label = driver.find_element(By.XPATH, "//label[contains(text(), '1 KG')]")
        kg_label.click()
        # Wait for the price to update
        time.sleep(2)
        page_soup = BeautifulSoup(driver.page_source, 'lxml')
        price_container = page_soup.find('div', class_='price')

        # Check if product is on sale
        if 'price--on-sale' in price_container['class']:
            # Get both regular and discount kg prices
            regular_price_kg = price_container.find('s', class_='price-item price-item--regular').text.strip()
            discount_price_kg = price_container.find('span', class_='price-item price-item--sale').text.strip()
        else:
            # Only get regular kg price, set discount kg price as empty
            regular_price_kg = price_container.find('span', class_='price-item price-item--regular').text.strip()
            discount_price_kg = ""

        kg_label.click()  # Click again to reset
        return regular_price_kg, discount_price_kg
    except:
        return "", ""

data = []
for url in urls:
    driver.get(url)

    handle_popup()

    slow_scroll_down(333)

    soup = BeautifulSoup(driver.page_source, 'lxml')
    products = soup.find_all('div', class_='card-information')

    for product in products:
        # Find product name
        name_element = product.find('span', class_='card-information__text h5')
        if name_element is None:
            continue
        name = name_element.text.strip()

        # Find product link and add base url
        link = "https://bazarstore.az" + product.find('a')['href']

        # Find price container
        price_container = product.find('div', class_='price')

        # Check if product is on sale
        if 'price--on-sale' in price_container['class']:
            regular_price = price_container.find('s', class_='price-item price-item--regular').text.strip()
            sale_price = price_container.find('span', class_='price-item price-item--sale').text.strip()
        else:
            regular_price = price_container.find('span', class_='price-item price-item--regular').text.strip()
            sale_price = ""

        # Navigate to product page
        driver.get(link)

        # Wait a bit for the page to load
        time.sleep(3)

        # Scrape the additional info
        page_soup = BeautifulSoup(driver.page_source, 'lxml')
        full_name = page_soup.find('h1', class_='product__title').text.strip()
        brand = page_soup.find('span', class_='product-vendor_name').find_next_sibling('a').text.strip()
        category = page_soup.find('span', class_='product-type').find_next_sibling(text=True).strip()
        sku = page_soup.find('span', class_='sku-name').find_next_sibling('span').text.strip()

        # Check if name and full name are the same, if not, replace name with full name
        if name != full_name:
            name = full_name

        # Get KG price if available
        regular_kg_price, discount_kg_price = get_kg_price()

        data.append([name, regular_price, sale_price, link, brand, category, sku, regular_kg_price, discount_kg_price])

driver.quit()

df = pd.DataFrame(data, columns=['name','regular_price', 'sale_price', 'link', 'brand', 'category', 'sku', 'regular_kg_price', 'discount_kg_price'])
print(df)


                                         name         regular_price  \
0                   PİNAR PEYNİR LABNE 100 GR                2.80 ₼   
1    PREZİDENT DİLİMLİ MAAZDAM  PENDİR 150 QR                4.10 ₼   
2    HOCHLAND DİLİMLİ PENDİR ÇİZBURQER 150 QR                3.99 ₼   
3       OLDENBURGER GOUDA 48% YAGLI PENDIR KG                4.68 ₼   
4             GÖYÇAY SÜD ÇANAX PENDİRİ 500 GR                5.49 ₼   
5               QAYMAQ AZ DUZLU PENDİR 500 QR                8.55 ₼   
6                    ÜÇ İNƏK AĞ PENDİR 200 QR                2.50 ₼   
7                          MİLLA AĞ PENDİR KQ                2.25 ₼   
8     SVALYA LITOVSKIY PENDİR DILIM 45% 150QR                4.70 ₼   
9                      MILLA AG PENDIR 250 GR                2.99 ₼   
10   PRESIDENT RONDELE PENDIR EDVIYYAT 125 GR                4.10 ₼   
11              ATENA PENDİR 250 QR MOZARELLA                4.00 ₼   
12                  ATENA PENDİR 250 QR QOUDA                4.00 ₼   
13    

In [396]:
import warnings 
warnings.filterwarnings("ignore")
import numpy as np

#pd.set_option('display.max_rows', None)
df1=df.copy()
df1.drop_duplicates(subset='name',keep='last', inplace=True)
df1

,name,regular_price,sale_price,link,brand,category,sku,regular_kg_price,discount_kg_price
0,PİNAR PEYNİR LABNE 100 GR,2.80 ₼,2.39 ₼,https://bazarstore.az/en/products/pi%CC%87nar-peyni%CC%87r-labne-100-gr?_pos=1&_fid=0445217ef&_ss=c,PINAR,: Eridilmiş Pendir,30131891,,
1,PREZİDENT DİLİMLİ MAAZDAM PENDİR 150 QR,4.10 ₼,3.65 ₼,https://bazarstore.az/en/products/prezi%CC%87dent-di%CC%87li%CC%87mli%CC%87-maazdam-pendi%CC%87r-150-qr?_pos=2&_fid=0445217ef&_ss=c,PRESIDENT,: Özəl Pendirlər,30012657,,
2,HOCHLAND DİLİMLİ PENDİR ÇİZBURQER 150 QR,3.99 ₼,3.35 ₼,https://bazarstore.az/en/products/hochland-di%CC%87li%CC%87mli%CC%87-pendi%CC%87r-ci%CC%87zburqer-150-qr?_pos=3&_fid=0445217ef&_ss=c,HOCHLAND,: Eridilmiş Pendir,30012634,,
3,OLDENBURGER GOUDA 48% YAGLI PENDIR KG,4.68 ₼,,https://bazarstore.az/en/products/oldenburger-gouda-48-yagli-pendir-kg?_pos=4&_fid=0445217ef&_ss=c,OLDENBUGER,: Özəl Pendirlər,30122302-1000,18.70 ₼,
4,GÖYÇAY SÜD ÇANAX PENDİRİ 500 GR,5.49 ₼,,https://bazarstore.az/en/products/goycay-sud-canax-pendi%CC%87ri%CC%87-500-gr?_pos=5&_fid=0445217ef&_ss=c,MILK PRO,: Ağ Pendir Ədəd,30101690,,
...,...,...,...,...,...,...,...,...,...
820,MÖCÜZƏ QAT QAT 250 QR,1.41 ₼,,https://bazarstore.az/products/mocuz%C9%99-qat-qat-250-qr?_pos=3&_fid=6ce529cce&_ss=c,MOCUZE,:,30155173,,
821,MÖCÜZƏ AĞ XƏMİR YAĞI 250 QR,1.44 ₼,,https://bazarstore.az/products/mocuz%C9%99-ag-x%C9%99mi%CC%87r-yagi-250-qr?_pos=4&_fid=6ce529cce&_ss=c,MOCUZE,:,10003668,,
822,MÖCÜZƏ KƏRƏ LƏZZƏTLİ QARĞIDALI YAĞI 250Q,1.33 ₼,1.10 ₼,https://bazarstore.az/products/mocuz%C9%99-k%C9%99r%C9%99-l%C9%99zz%C9%99tli%CC%87-qargidali-yagi-250q?_pos=5&_fid=6ce529cce&_ss=c,MOCUZE,:,10000330,,
823,SANA YAĞI 250 QR PAKET,1.32 ₼,,https://bazarstore.az/products/sana-yagi-250-qr-paket?_pos=6&_fid=6ce529cce&_ss=c,SANA,:,10000328,,


In [397]:
# Load the necessary packages
import pandas as pd
import numpy as np
import re

# Define the clean_price_column function
def clean_price_column(column):
    column = column.str.replace('₼', '').str.replace('Başl. qiymət:', '').str.replace('Regular price', '')
    column = column.apply(lambda x: re.findall(r'\d+\.\d+', x)[0] if re.findall(r'\d+\.\d+', x) else x)
    column = column.replace('', np.nan)
    column = column.astype(float)
    return column


# Clean up the 'regular_price', 'sale_price', 'regular_kg_price', and 'discount_kg_price' columns
df1["regular_price"] = clean_price_column(df1["regular_price"])
df1["sale_price"] = clean_price_column(df1["sale_price"])
df1["regular_kg_price"] = clean_price_column(df1["regular_kg_price"])
df1["discount_kg_price"] = clean_price_column(df1["discount_kg_price"])

# For rows where regular_kg_price is not NaN, copy the corresponding regular_kg_price value to regular_price
df1.loc[~df1["regular_kg_price"].isnull(), "regular_price"] = df1["regular_kg_price"]

# For rows where discount_kg_price is not NaN, copy the corresponding discount_kg_price value to sale_price
df1.loc[~df1["discount_kg_price"].isnull(), "sale_price"] = df1["discount_kg_price"]

# Re-calculate the discount rate
df1["endirim_faiz"] = (df1["regular_price"] - df1["sale_price"]) / df1["regular_price"]

# Clean up the 'category' column
df1["category"] = df1["category"].str.replace(':', '')

# Format 'endirim_faiz' to 4 decimal places and convert to percentage
df1["endirim_faiz"] = (df1["endirim_faiz"] * 100).round(2)

# Drop the 'regular_kg_price' and 'discount_kg_price' columns
df1.drop(columns=['regular_kg_price', 'discount_kg_price'], inplace=True)

# Modify the 'name' column to add '1000 ' in front of 'KG' and 'KQ' if there is no other number before 'KG' or 'KQ'
df1['name'] = df1['name'].apply(lambda x: re.sub(r'(?<=\s)(KG|KQ)', r'1000 \1', x))

# Define the regular expression pattern
pattern = r'(\d+\.*\d*\s*[GQRKgqrkMLLT+Rr]+)'

# Apply the pattern to extract the weight from the 'name' column
df1['qramaj'] = df1['name'].str.extract(pattern)[0]

# Fill NA values with a default string, let's say 'NA'
df1['qramaj'] = df1['qramaj'].fillna('NA')

# Now we handle the conversion for each unit
df1.loc[df1['qramaj'].str.contains('G|g', na=False), 'qramaj'] = df1.loc[df1['qramaj'].str.contains('G|g', na=False), 'qramaj'].str.replace(r'\D+', '').astype(float)
df1.loc[df1['qramaj'].str.contains('QR|qr', na=False), 'qramaj'] = df1.loc[df1['qramaj'].str.contains('QR|qr', na=False), 'qramaj'].str.replace(r'\D+', '').astype(float)
df1.loc[df1['qramaj'].str.contains('KG|KQ|kg|kq', na=False), 'qramaj'] = df1.loc[df1['qramaj'].str.contains('KG|KQ|kg|kq', na=False), 'qramaj'].str.replace(r'\D+', '').astype(float)
df1.loc[df1['qramaj'].str.contains('ML|ml', na=False), 'qramaj'] = df1.loc[df1['qramaj'].str.contains('ML|ml', na=False), 'qramaj'].str.replace(r'\D+', '').astype(float)
df1.loc[df1['qramaj'].str.contains('LT|lt', na=False), 'qramaj'] = df1.loc[df1['qramaj'].str.contains('LT|lt', na=False), 'qramaj'].str.replace(r'\D+', '').astype(float) * 1000
df1.loc[df1['qramaj'].str.contains('\+|R|r', na=False), 'qramaj'] = df1.loc[df1['qramaj'].str.contains('\+|R|r', na=False), 'qramaj'].str.replace(r'\D+', '').astype(float)

# Handle 'KG' and 'KQ' without a preceding numeric value separately
df1.loc[df1['qramaj'] == 'NA', 'qramaj'] = df1.loc[df1['qramaj'] == 'NA', 'name'].apply(lambda x: 1000 if 'KG' in x or 'KQ' in x else 'NA')

# Extracting fat percentage
df1['yag_faiz'] = df1['name'].str.extract(r'(\d+[\.,]?\d*\s*%)')[0].str.replace('%', '').str.replace(',', '.').astype(float)

# Replace NaNs in 'yag_faiz' with empty
df1['yag_faiz'] = df1['yag_faiz'].replace(np.nan, '')


In [398]:
# Set category to 'Marqarin' for specified links
links = ["https://bazarstore.az/products/mocuz%C9%99-k%C9%99r%C9%99-l%C9%99zz%C9%99tli%CC%87-500-gr-qargidali?_pos=1&_fid=834af9c03&_ss=c",
"https://bazarstore.az/products/mocuz%C9%99-krem-yagi-250-qr?_pos=2&_fid=834af9c03&_ss=c",
"https://bazarstore.az/products/mocuz%C9%99-qat-qat-250-qr?_pos=3&_fid=834af9c03&_ss=c",
"https://bazarstore.az/products/mocuz%C9%99-ag-x%C9%99mi%CC%87r-yagi-250-qr?_pos=4&_fid=834af9c03&_ss=c",
"https://bazarstore.az/products/mocuz%C9%99-k%C9%99r%C9%99-l%C9%99zz%C9%99tli%CC%87-qargidali-yagi-250q?_pos=5&_fid=834af9c03&_ss=c",
"https://bazarstore.az/products/fi%CC%87nal-bi%CC%87tki%CC%87-yagi-250-qr?_pos=6&_fid=834af9c03&_ss=c"]

for link in links:
    df1.loc[df1['link'] == link, 'category'] = 'Marqarin'

# Convert all category values to upper case
df1['category'] = df1['category'].str.upper()

In [399]:
correction_dict = {
    "https://bazarstore.az/products/prezi%CC%87dent-di%CC%87li%CC%87mli%CC%87-mozarella-pendi%CC%87r-150?_pos=15&_fid=5da8ae25e&_ss=c": 150,
    "https://bazarstore.az/products/savuski%CC%87n-brest-li%CC%87tovsk-k%C9%99si%CC%87k-pendi%CC%87r-45?_pos=28&_fid=9cc41f453&_ss=c": 210,
    "https://bazarstore.az/products/brest-li%CC%87tovsk-k%C9%99r%C9%99-yagli-pendi%CC%87r-50-210q?_pos=34&_fid=9cc41f453&_ss=c": 210,
    "https://bazarstore.az/products/i%CC%87calki-yantar-pendi%CC%87r-80-q?_pos=80&_fid=9cc41f453&_ss=c": 80,
    "https://bazarstore.az/products/savuski%CC%87n-pendi%CC%87r-brest-li%CC%87tovsk-leqk-na-35?_pos=87&_fid=9cc41f453&_ss=c": 210,
    "https://bazarstore.az/products/savuski%CC%87n-pendi%CC%87r-brest-li%CC%87tovsk-klas-na-45?_pos=88&_fid=9cc41f453&_ss=c": 210,
    "https://bazarstore.az/products/emborg-pendir-grana-padano-32?_pos=114&_fid=9cc41f453&_ss=c": 150,
    "https://bazarstore.az/products/savuski%CC%87n-pendi%CC%87r-rossiyskiy-k%C9%99smi%CC%87k-50-20?_pos=123&_fid=16790766a&_ss=c": 200,
    "https://bazarstore.az/products/tahsi%CC%87ldaroglu-kohne-kasar-350-q-%C9%99zi%CC%87n%C9%99?_pos=145&_fid=16790766a&_ss=c": 350,
    "https://bazarstore.az/products/tahsildaroglu-ezine-klasik-ag-pendi%CC%87r-600?_pos=190&_fid=16790766a&_ss=c": 600,
    "https://bazarstore.az/products/s%C9%99ba-k%C9%99r%C9%99-yagi-82-1kq?_pos=4&_fid=db4c1d2df&_ss=c" : 1000,
    "https://bazarstore.az/products/yag-korovuska-82-5-1kg?_pos=41&_fid=db4c1d2df&_ss=c" : 1000,
    "https://bazarstore.az/products/yeni%CC%87-zellandi%CC%87ya-k%C9%99r%C9%99-yagi-newland-1kq?_pos=74&_fid=ef05db37b&_ss=c" : 1000,
    "https://bazarstore.az/products/cey-cey-sokolad-2-r%C9%99ng-350-qr?_pos=269&_fid=7de4a0386&_ss=c" : 350,
    "https://bazarstore.az/products/kalinka-kereli-yag-2kg?_pos=38&_fid=db4c1d2df&_ss=c" : 2000,
    "https://bazarstore.az/products/kido-muzlu-sut-1-5-lt?_pos=191&_fid=9ff7f5fe5&_ss=c" : 200,
    "https://bazarstore.az/products/kido-cilekli-sut-1-5-lt?_pos=192&_fid=9ff7f5fe5&_ss=c" : 200,
    "https://bazarstore.az/products/ki%CC%87do-kakaolu-sud-1-5-lt?_pos=193&_fid=9ff7f5fe5&_ss=c": 200,
    "https://bazarstore.az/en/products/prezi%CC%87dent-di%CC%87li%CC%87mli%CC%87-mozarella-pendi%CC%87r-150?_pos=15&_fid=0445217ef&_ss=c": 150,
    "https://bazarstore.az/en/products/savuski%CC%87n-brest-li%CC%87tovsk-k%C9%99si%CC%87k-pendi%CC%87r-45?_pos=28&_fid=7bc24e166&_ss=c": 150,
    "https://bazarstore.az/en/products/brest-li%CC%87tovsk-k%C9%99r%C9%99-yagli-pendi%CC%87r-50-210q?_pos=35&_fid=7bc24e166&_ss=c": 210,
    "https://bazarstore.az/en/products/i%CC%87calki-yantar-pendi%CC%87r-80-q?_pos=82&_fid=ba67ef3b2&_ss=c": 80,
    "https://bazarstore.az/en/products/savuski%CC%87n-pendi%CC%87r-brest-li%CC%87tovsk-leqk-na-35?_pos=89&_fid=ba67ef3b2&_ss=c": 350,
    "https://bazarstore.az/en/products/savuski%CC%87n-pendi%CC%87r-brest-li%CC%87tovsk-klas-na-45?_pos=90&_fid=ba67ef3b2&_ss=c": 450,
    "https://bazarstore.az/en/products/emborg-pendir-grana-padano-32?_pos=115&_fid=ba67ef3b2&_ss=c": 150,
    "https://bazarstore.az/en/products/savuski%CC%87n-pendi%CC%87r-rossiyskiy-k%C9%99smi%CC%87k-50-20?_pos=124&_fid=ba67ef3b2&_ss=c": 200,
    "https://bazarstore.az/en/products/tahsi%CC%87ldaroglu-kohne-kasar-350-q-%C9%99zi%CC%87n%C9%99?_pos=146&_fid=ad21041b8&_ss=c": 350,
    "https://bazarstore.az/en/products/tahsildaroglu-ezine-klasik-ag-pendi%CC%87r-600?_pos=190&_fid=ad21041b8&_ss=c": 600,
    "https://bazarstore.az/products/atena-ayran-1l?_pos=2&_fid=4c5398bd4&_ss=c": 1000,
    "https://bazarstore.az/products/az%C9%99rsud-qaymaq-26-200-qr?_pos=3&_fid=4c5398bd4&_ss=c": 200,
    "https://bazarstore.az/products/prostokvashino-si%CC%87ri%CC%87n-k%C9%99smik-iris-40-gr?_pos=14&_fid=4c5398bd4&_ss=c": 40,
    "https://bazarstore.az/products/prostokvasi%CC%87no-si%CC%87rok-qatilasdirilmis-sud?_pos=17&_fid=4c5398bd4&_ss=c": 40,
    "https://bazarstore.az/products/prostokvasi%CC%87no-qaymaq-205-gr-10-duru?_pos=21&_fid=4c5398bd4&_ss=c": 205,
    "https://bazarstore.az/products/prostokvasi%CC%87no-qaymaq-200-gr-20-duru?_pos=22&_fid=4c5398bd4&_ss=c": 200,
    "https://bazarstore.az/products/yurdum-ayran-1-l?_pos=25&_fid=e6ab855d1&_ss=c": 1000,
    "https://bazarstore.az/products/doktor-mi%CC%87lk-k%C9%99smi%CC%87k-ci%CC%87y%C9%99l%C9%99k-fruk-0-180?_pos=30&_fid=e6ab855d1&_ss=c": 180,
    "https://bazarstore.az/products/ivanovka-uzun-omur-erikli-kesmik-18-400?_pos=34&_fid=e6ab855d1&_ss=c": 400,
    "https://bazarstore.az/products/president-kofe-qaymaq-100-gr?_pos=36&_fid=e6ab855d1&_ss=c": 100,
    "https://bazarstore.az/products/prostokvasi%CC%87no-si%CC%87rok-vani%CC%87l-40-qr?_pos=37&_fid=e6ab855d1&_ss=c": 40,
    "https://bazarstore.az/products/ulk%C9%99r-qaymaq-200ml?_pos=39&_fid=e6ab855d1&_ss=c": 200,
    "https://bazarstore.az/products/uvelka-4-taxilli-tonki%CC%87e-xlopya-350gr?_pos=14&_fid=46cd293cc&_ss=c": 350,
    "https://bazarstore.az/products/bi%CC%87zi%CC%87m-tarla-qirmizi-gi%CC%87las-mur%C9%99bb%C9%99si%CC%87-400?_pos=33&_fid=ea8ccec88&_ss=c": 400,
    "https://bazarstore.az/products/kelloggs-granola-340-q-sokolad-findiqli?_pos=59&_fid=ea8ccec88&_ss=c": 340,
    "https://bazarstore.az/products/uvelka-4-taxilli-xlopya-350gr?_pos=113&_fid=f6b8c3661&_ss=c": 350,
    "https://bazarstore.az/products/uvelka-xlopya-6-taxilli-350gr?_pos=114&_fid=f6b8c3661&_ss=c": 350,
    "https://bazarstore.az/products/qi%CC%87nyetogullari-cok%C9%99-bali-470-q?_pos=116&_fid=f6b8c3661&_ss=c": 470,
    "https://bazarstore.az/products/qi%CC%87nyetogullari-dag-bali-470-q?_pos=117&_fid=f6b8c3661&_ss=c": 470,
    "https://bazarstore.az/products/qlavpodukt-qatilasdirilmis-sud-ekstra-38?_pos=118&_fid=f6b8c3661&_ss=c": 380,
    "https://bazarstore.az/products/car-yulaf-6-taxil-400-q?_pos=119&_fid=f6b8c3661&_ss=c": 400,
    "https://bazarstore.az/products/car-yulaf-lopalari-400-q-ekstra-1?_pos=120&_fid=f6b8c3661&_ss=c": 400,
    "https://bazarstore.az/products/car-yulaf-lopalari-400-q-paket-qerkules?_pos=121&_fid=f6b8c3661&_ss=c": 400,
    "https://bazarstore.az/products/lifalif-hurmali-ve-yer-fistikli-musli-35?_pos=163&_fid=f6b8c3661&_ss=c": 350,
    "https://bazarstore.az/products/feel-true-fistig-ezmesi-sekersiz-tebii-l?_pos=166&_fid=f6b8c3661&_ss=c": 270,
    "https://bazarstore.az/products/nestle-bi%CC%87strov-assorti%CC%87-yulaf-siyigi-210?_pos=168&_fid=f6b8c3661&_ss=c": 210,
    "https://bazarstore.az/products/russkiy-produkt-qerkules-qaragi%CC%87l%C9%99-moruq?_pos=169&_fid=f6b8c3661&_ss=c": 420,
    "https://bazarstore.az/products/russkiy-produkt-qerkules-ci%CC%87y%C9%99l%C9%99k-albali?_pos=170&_fid=f6b8c3661&_ss=c": 420,
    "https://bazarstore.az/products/russkiy-produkt-qerkules-5-taxilli-400-g?_pos=171&_fid=f6b8c3661&_ss=c": 400,
    "https://bazarstore.az/products/eti%CC%87-li%CC%87fali%CC%87f-musli%CC%87-meyv%C9%99li%CC%87-findiqli-350-q?_pos=203&_fid=7de4a0386&_ss=c": 350,
    "https://bazarstore.az/products/korenovskaya-bi%CC%87smi%CC%87s-qatilasdirilmis-sud?_pos=235&_fid=7de4a0386&_ss=c": 370,
    "https://bazarstore.az/products/korenovskaya-qatilasdirilmis-sud-8-5-tn?_pos=236&_fid=7de4a0386&_ss=c": 370,
    "https://bazarstore.az/products/car-yulaf-7-taxil-400-gr?_pos=246&_fid=7de4a0386&_ss=c": 400,
    "https://bazarstore.az/products/kuli%CC%87narka-qatilasdirilmis-sud-s%C9%99k%C9%99rli%CC%87-37?_pos=247&_fid=7de4a0386&_ss=c": 370,
    "https://bazarstore.az/products/russki%CC%87y-produkt-qerkules-tradi%CC%87si%CC%87onni%CC%87-420?_pos=248&_fid=7de4a0386&_ss=c": 420,
    "https://bazarstore.az/products/kuli%CC%87narka-kakao-qatilasdirilmis-sud-s%C9%99k%C9%99?_pos=268&_fid=7de4a0386&_ss=c": 370,
    "https://bazarstore.az/products/kuli%CC%87narka-bi%CC%87smi%CC%87s-qatilasdirilmis-sud-370?_pos=273&_fid=7de4a0386&_ss=c": 370,
    "https://bazarstore.az/products/kuli%CC%87narka-qatilasdirilmis-sud-s%C9%99k%C9%99rli%CC%87-95?_pos=274&_fid=7de4a0386&_ss=c": 950,
    "https://bazarstore.az/products/bizim-tarla-mur%C9%99bb%C9%99-400-q-%C9%99nci%CC%87r?_pos=286&_fid=7de4a0386&_ss=c": 400,
    "https://bazarstore.az/products/atena-sud-3-2-1l?_pos=4&_fid=3efbd478c&_ss=c": 1000,
    "https://bazarstore.az/products/rasti%CC%87ska-k%C9%99smi%CC%87k-ci%CC%87y%C9%99l%C9%99k-banan-plombi%CC%87r?_pos=8&_fid=3efbd478c&_ss=c": 270,
    "https://bazarstore.az/products/akti%CC%87vi%CC%87a-i%CC%87cm%C9%99li%CC%87-yoqurt-qaragil%C9%99-taxillar?_pos=12&_fid=3efbd478c&_ss=c": 260,
    "https://bazarstore.az/products/alpro-badam-1l?_pos=13&_fid=3efbd478c&_ss=c": 1000,
    "https://bazarstore.az/products/danone-rasti%CC%87ska-yogurt-moruq-banan-110-q?_pos=14&_fid=3efbd478c&_ss=c": 110,
    "https://bazarstore.az/products/az%C9%99r-sud-balans-i%CC%87mmuni%CC%87tet-sudu-1l-3-2?_pos=15&_fid=3efbd478c&_ss=c": 1000,
    "https://bazarstore.az/products/mi%CC%87lla-yoqurt-115-q-gi%CC%87l%C9%99meyv%C9%99?_pos=16&_fid=3efbd478c&_ss=c": 1150,
    "https://bazarstore.az/products/mi%CC%87lla-yoqurt-115-q-saftali?_pos=17&_fid=3efbd478c&_ss=c": 115,
    "https://bazarstore.az/products/qlavpodukt-qat-sud-380-q-yubi%CC%87leynaya?_pos=22&_fid=3efbd478c&_ss=c": 380,
    "https://bazarstore.az/products/atena-sud-1-5-1l?_pos=24&_fid=3efbd478c&_ss=c": 1000,
    "https://bazarstore.az/products/alpro-badam-s%C9%99k%C9%99rsi%CC%87z-1l?_pos=31&_fid=564a68e6d&_ss=c": 1000,
    "https://bazarstore.az/products/milla-2-5-sud-1-l?_pos=33&_fid=564a68e6d&_ss=c": 1000,
    "https://bazarstore.az/products/akti%CC%87mel-qaragi%CC%87l%C9%99-boyurtk%C9%99n-i%CC%87cm%C9%99li%CC%87-yogurt?_pos=39&_fid=564a68e6d&_ss=c": 95,
    "https://bazarstore.az/products/mi%CC%87lla-yoqurt-115-q-gi%CC%87las?_pos=41&_fid=564a68e6d&_ss=c": 115,
    "https://bazarstore.az/products/ulker-cocostar-cvz-5li_125g-baton?_pos=64&_fid=564a68e6d&_ss=c": 125,
    "https://bazarstore.az/products/savuski%CC%87n-yogurt-laska-yay-ci%CC%87y%C9%99l%C9%99yi%CC%87-2-12?_pos=67&_fid=564a68e6d&_ss=c": 120,
    "https://bazarstore.az/products/grand-duo-yoqurt-230-sokolad-vani%CC%87lli%CC%87?_pos=81&_fid=564a68e6d&_ss=c": 230,
    "https://bazarstore.az/products/aktiviya-yoqurt-saftali-li%CC%87ce-ki%CC%87noa-130-q?_pos=107&_fid=564a68e6d&_ss=c": 130,
    "https://bazarstore.az/products/akti%CC%87vi%CC%87a-i%CC%87cm%C9%99li%CC%87-yoqurt-qara-gavali-260-q?_pos=118&_fid=564a68e6d&_ss=c": 260,
    "https://bazarstore.az/products/aktivia-icm%C9%99li-yoqurt-yer-ciy%C9%99l%C9%99yi-ciy%C9%99l?_pos=119&_fid=564a68e6d&_ss=c": 260,
    "https://bazarstore.az/products/alpro-kokos-duyu-1l?_pos=124&_fid=9ff7f5fe5&_ss=c": 1000,
    "https://bazarstore.az/products/pi%CC%87nar-sud-protei%CC%87n-y-fi%CC%87s-ve-muz-aromali%CC%87-5?_pos=125&_fid=9ff7f5fe5&_ss=c": 500,
    "https://bazarstore.az/products/danone-rasti%CC%87ska-i%CC%87cm%C9%99li%CC%87-saftali-banan-90q?_pos=137&_fid=9ff7f5fe5&_ss=c": 90,
    "https://bazarstore.az/products/dani%CC%87ssi%CC%87mo-yogurt-gi%CC%87l%C9%99meyv%C9%99-d%C9%99n%C9%99l%C9%99ri%CC%87-105q?_pos=140&_fid=9ff7f5fe5&_ss=c": 105,
    "https://bazarstore.az/products/cudo-yoqurt-saftali-mango-yemi%CC%87s-2-4-290?_pos=168&_fid=9ff7f5fe5&_ss=c": 290,
    "https://bazarstore.az/products/mi%CC%87lla-yoqurt-115-q-ci%CC%87y%C9%99l%C9%99k?_pos=169&_fid=9ff7f5fe5&_ss=c": 115,
    "https://bazarstore.az/products/dani%CC%87ssi%CC%87mo-fantazi%CC%87ya-xircildayan-kur%C9%99ci%CC%87k?_pos=181&_fid=9ff7f5fe5&_ss=c": 105,
    "https://bazarstore.az/products/domi%CC%87k-derevne-sud-6-1-l?_pos=189&_fid=9ff7f5fe5&_ss=c": 1000,
    "https://bazarstore.az/products/super-mi%CC%87lky-sokoladli-sud-1-l?_pos=195&_fid=9ff7f5fe5&_ss=c": 1000,
    "https://bazarstore.az/products/super-mi%CC%87lky-ci%CC%87y%C9%99l%C9%99kli-sud-1-l?_pos=196&_fid=9ff7f5fe5&_ss=c": 1000,
    "https://bazarstore.az/products/mocuz%C9%99-k%C9%99r%C9%99-l%C9%99zz%C9%99tli%CC%87-qargidali-yagi-250q?_pos=5&_fid=6ce529cce&_ss=c": 250,
    "https://bazarstore.az/products/nesquik-qargidali-lopalari-450-q?_pos=253&_fid=7de4a0386&_ss=c" : 450,
    "https://bazarstore.az/en/products/pinar-labne-multi%CC%87pak-pendi%CC%87r-8x20-qr?_pos=149&_fid=ad21041b8&_ss=c" : 160,
    "https://bazarstore.az/products/fi%CC%87nal-m%C9%99tb%C9%99x-yagi-1-8-kg-200-gr-1?_pos=12&_fid=db4c1d2df&_ss=c" : 1200,
    "https://bazarstore.az/products/mi%CC%87lla-yoqurt-115-q-gi%CC%87l%C9%99meyv%C9%99?_pos=16&_fid=3efbd478c&_ss=c" : 115
}




# Apply the correction
df1.loc[df1['link'].isin(correction_dict.keys()), 'qramaj'] = df1.loc[df1['link'].isin(correction_dict.keys()), 'link'].map(correction_dict)

In [400]:
df1['yag_faiz'][df1.link == "https://bazarstore.az/products/dr-milk-kaunas-xama-200-gr-10?_pos=30&_fid=4a3e11d90&_ss=c"] = "10"
df1['yag_faiz'][df1.link == "https://bazarstore.az/en/products/azersud-100-tebii-antep-pendiri-300-gr?_pos=73&_fid=ba67ef3b2&_ss=c"] = ""

df1['brand'][df1.link == "https://bazarstore.az/en/products/azersud-ag-pendi%CC%87r-400-qr?_pos=105&_fid=ba67ef3b2&_ss=c"] = "AZERSUD"
df1['brand'][df1.link == "https://bazarstore.az/products/azersud-kesmik-yumsaq-230-gr?_pos=13&_fid=4c5398bd4&_ss=c"] = "AZERSUD"
df1['brand'][df1.link == "https://bazarstore.az/products/azersud-yurdum-k%C9%99smi%CC%87k-kg?_pos=1&_fid=4c5398bd4&_ss=c"] = "YURDUM"
df1['brand'][df1.link == "https://bazarstore.az/en/products/atena-pendi%CC%87r-250-qr-mozarella?_pos=12&_fid=0445217ef&_ss=c"] = "ATENA"
df1['brand'][df1.link == "https://bazarstore.az/products/best-cow-k%C9%99r%C9%99-yagi-1-kq-1?_pos=78&_fid=ef05db37b&_ss=c"] = "BEST COW"
df1['brand'][df1.link == "https://bazarstore.az/products/best-cow-k%C9%99r%C9%99-yagi-200-qr-1?_pos=77&_fid=ef05db37b&_ss=c"] = "BEST COW"
df1['brand'][df1.link == "https://bazarstore.az/products/car-gerkules-400-qr?_pos=100&_fid=f6b8c3661&_ss=c"] = "TSAR"
df1['brand'][df1.link == "https://bazarstore.az/products/dani%CC%87ssi%CC%87mo-fantazi%CC%87ya-xircildayan-kur%C9%99ci%CC%87k?_pos=181&_fid=9ff7f5fe5&_ss=c"] = "DANISSIMO"
df1['brand'][df1.link == "https://bazarstore.az/products/dani%CC%87ssi%CC%87mo-yogurt-gi%CC%87l%C9%99meyv%C9%99-d%C9%99n%C9%99l%C9%99ri%CC%87-105q?_pos=140&_fid=9ff7f5fe5&_ss=c"] = "DANISSIMO"
df1['brand'][df1.link == "https://bazarstore.az/products/doktor-mi%CC%87lk-k%C9%99smi%CC%87k-180-gr-albali-0?_pos=6&_fid=4c5398bd4&_ss=c"] = "DR.MILK"
df1['brand'][df1.link == "https://bazarstore.az/products/doktor-mi%CC%87lk-k%C9%99smi%CC%87k-ci%CC%87y%C9%99l%C9%99k-fruk-0-180?_pos=30&_fid=e6ab855d1&_ss=c"] = "DR.MILK"
df1['brand'][df1.link == "https://bazarstore.az/products/doktor-mi%CC%87lk-k%C9%99smi%CC%87k-180-gr-albali-0?_pos=6&_fid=4c5398bd4&_ss=c"] = "DR.MILK"
df1['brand'][df1.link == "https://bazarstore.az/products/doktor-milk-qatiq-450-gr-4-6?_pos=16&_fid=ed8c9b170&_ss=c"] = "DR.MILK"
df1['brand'][df1.link == "https://bazarstore.az/en/products/doktor-milk-urek-pendir-kg?_pos=42&_fid=7bc24e166&_ss=c"] = "DR.MILK"
df1['brand'][df1.link == "https://bazarstore.az/en/products/doktor-milk-urek-pendir-kg?_pos=42&_fid=7bc24e166&_ss=c"] = "DR.MILK"

df1['brand'][df1.link == "https://bazarstore.az/products/eti-sut-burger-ciyelek-35gr?_pos=185&_fid=f6b8c3661&_ss=c"] = "ETI"
df1['brand'][df1.link == "https://bazarstore.az/products/eti-burcak-kakao-kremali-82-qr?_pos=184&_fid=f6b8c3661&_ss=c"] = "ETI"
df1['brand'][df1.link == "https://bazarstore.az/products/eti-gong-pops-orijinal-80-qr?_pos=181&_fid=f6b8c3661&_ss=c"] = "ETI"
df1['brand'][df1.link == "https://bazarstore.az/products/eti%CC%87-li%CC%87fali%CC%87f-musli%CC%87-meyv%C9%99li%CC%87-findiqli-350-q?_pos=203&_fid=7de4a0386&_ss=c"] = "ETI"
df1['brand'][df1.link == "https://bazarstore.az/products/eti-popkek-45-qr-portagalli?_pos=182&_fid=f6b8c3661&_ss=c"] = "ETI"
df1['brand'][df1.link == "https://bazarstore.az/en/products/galbani-mascarpone-250-gr?_pos=24&_fid=0445217ef&_ss=c"] = "GALBANI"
df1['brand'][df1.link == "https://bazarstore.az/products/gerda-alman-k%C9%99r%C9%99-yagi-kq?_pos=5&_fid=db4c1d2df&_ss=c"] = "GERDA"
df1['brand'][df1.link == "https://bazarstore.az/products/gerda-alman-k%C9%99r%C9%99-yagi-250qr?_pos=51&_fid=db4c1d2df&_ss=c"] = "GERDA"

df1['brand'][df1.link == "https://bazarstore.az/products/grand-cocktail-duzlu-karamel-4-260gr?_pos=130&_fid=9ff7f5fe5&_ss=c"] = "EHRMANN"
df1['brand'][df1.link == "https://bazarstore.az/products/grand-cocktail-vanilli-plombir-4-260gr?_pos=128&_fid=9ff7f5fe5&_ss=c"] = "EHRMANN"
df1['brand'][df1.link == "https://bazarstore.az/products/grand-dessert-ikiqat-findiq-4-9-200gr?_pos=129&_fid=9ff7f5fe5&_ss=c"] = "EHRMANN"
df1['brand'][df1.link == "https://bazarstore.az/en/products/i%CC%87ci%CC%87m-pendi%CC%87r-ucbucaq-100-gr?_pos=106&_fid=ba67ef3b2&_ss=c"] = "DR.MILK"
df1['brand'][df1.link == "https://bazarstore.az/products/icim-puding-shok-findiq-60-gr?_pos=88&_fid=564a68e6d&_ss=c"] = "DR.MILK"
df1['brand'][df1.link == "https://bazarstore.az/products/ivanovka-uzun-omur-erikli-kesmik-18-400?_pos=34&_fid=e6ab855d1&_ss=c"] = "UZUN OMUR"
df1['brand'][df1.link == "https://bazarstore.az/products/kali%CC%87nka-bi%CC%87tki%CC%87-yag-spredi%CC%87-82-200-qr?_pos=76&_fid=ef05db37b&_ss=c"] = "KALINKA"
df1['brand'][df1.link == "https://bazarstore.az/products/keks-ozmo-mello-30gr?_pos=186&_fid=f6b8c3661&_ss=c"] = "OZMO"
df1['brand'][df1.link == "https://bazarstore.az/products/kelloggs-granola-340-q-sokolad-findiqli?_pos=59&_fid=ea8ccec88&_ss=c"] = "KELLOGGS"
df1['brand'][df1.link == "https://bazarstore.az/products/luppo-cake-300-qr-bi%CC%87tter-dark?_pos=187&_fid=f6b8c3661&_ss=c"] = "LUPPO"
df1['brand'][df1.link == "https://bazarstore.az/en/products/milla-labne-pendir-400-gr?_pos=103&_fid=ba67ef3b2&_ss=c"] = "MILLA"
df1['brand'][df1.link == "https://bazarstore.az/en/products/n1-pendi%CC%87r-holland-250-gr?_pos=109&_fid=ba67ef3b2&_ss=c"] = "N1"
df1['brand'][df1.link == "https://bazarstore.az/en/products/n1-pendi%CC%87r-mozarella-250-gr?_pos=110&_fid=ba67ef3b2&_ss=c"] = "N1"
df1['brand'][df1.link == "https://bazarstore.az/en/products/pendir-atena-canaq-kg?_pos=66&_fid=e771fcc14&_ss=c"] = "ATENA"

df1['brand'][df1.link == "https://bazarstore.az/en/products/pendir-hochland-150gr-kesme-dilim?_pos=114&_fid=ba67ef3b2&_ss=c"] = "HOCHLAND"
df1['brand'][df1.link == "https://bazarstore.az/en/products/pinar-labne-firsat-paketi-200-gr-3-lu?_pos=31&_fid=7bc24e166&_ss=c"] = "PINAR"
df1['brand'][df1.link == "https://bazarstore.az/products/prostokvashino-si%CC%87ri%CC%87n-k%C9%99smik-iris-40-gr?_pos=14&_fid=4c5398bd4&_ss=c"] = "PROSTOKVASINO"
df1['brand'][df1.link == "https://bazarstore.az/products/seba-k%C9%99nd-qatigi-2-lt?_pos=4&_fid=ed8c9b170&_ss=c"] = "SEBA"
df1['brand'][df1.link == "https://bazarstore.az/products/shokki-mokki-sokoladli-sud-200-ml?_pos=29&_fid=564a68e6d&_ss=c"] = "SHOKKI MOKKI"

df1['brand'][df1.link == "https://bazarstore.az/products/start-si%CC%87r%C9%99l%C9%99nmi%CC%87s-qargida-lopalari-90-qr?_pos=101&_fid=f6b8c3661&_ss=c"] = "START"
df1['brand'][df1.link == "https://bazarstore.az/products/super-mi%CC%87lky-kakaolu-sud-200-ml-tetra-pak-1?_pos=197&_fid=9ff7f5fe5&_ss=c"] = "AZERSUD"
df1['brand'][df1.link == "https://bazarstore.az/products/vali%CC%87o-k%C9%99r%C9%99-yagi-500qr?_pos=58&_fid=db4c1d2df&_ss=c"] = "VALIO"
df1['brand'][df1.link == "https://bazarstore.az/products/yeni%CC%87-zellandi%CC%87ya-k%C9%99r%C9%99-yagi-newland-1kq?_pos=74&_fid=ef05db37b&_ss=c"] = "NEWLAND"
df1['brand'][df1.link == "https://bazarstore.az/products/krestyanskoe-82-kq?_pos=23&_fid=db4c1d2df&_ss=c"] = "KRESTYANSKOE"
df1['brand'][df1.link == "https://bazarstore.az/products/seba-k%C9%99nd-qatigi-2-lt?_pos=4&_fid=ed8c9b170&_ss=c"] = "SEBA"
df1['brand'][df1.link == "https://bazarstore.az/products/seba-k%C9%99nd-qatigi-2-lt?_pos=4&_fid=ed8c9b170&_ss=c"] = "SEBA"



In [401]:
# Define a list of URLs to be updated
urls_to_update = [
    "https://bazarstore.az/products/fi%CC%87nal-bi%CC%87tki%CC%87-yagi-250-qr?_pos=7&_fid=6ce529cce&_ss=c",
    "https://bazarstore.az/products/mocuz%C9%99-k%C9%99r%C9%99-l%C9%99zz%C9%99tli%CC%87-500-gr-qargidali?_pos=1&_fid=6ce529cce&_ss=c",
    "https://bazarstore.az/products/mocuz%C9%99-krem-yagi-250-qr?_pos=2&_fid=6ce529cce&_ss=c",
    "https://bazarstore.az/products/mocuz%C9%99-qat-qat-250-qr?_pos=3&_fid=6ce529cce&_ss=c",
    "https://bazarstore.az/products/mocuz%C9%99-ag-x%C9%99mi%CC%87r-yagi-250-qr?_pos=4&_fid=6ce529cce&_ss=c",
    "https://bazarstore.az/products/mocuz%C9%99-k%C9%99r%C9%99-l%C9%99zz%C9%99tli%CC%87-qargidali-yagi-250q?_pos=5&_fid=6ce529cce&_ss=c",
    "https://bazarstore.az/products/sana-yagi-250-qr-paket?_pos=6&_fid=6ce529cce&_ss=c"
]

# Update the 'category' column for the specified URLs
df1.loc[df['link'].isin(urls_to_update), 'category'] = "MELANJ YAĞI (ƏDƏD)"

# Define a list of groups to be deleted
groups_to_delete = ["BAL", "BATON ŞOKOLAD", "BƏHMƏZ", "CEM", "HALVA", "KREM ƏZMƏLƏR", "MÜRƏBBƏ", "POVIDLO", "SƏHƏR YEMƏKLƏRI", "TAHIN"]

df1['category'] = df1['category'].str.strip().str.upper()

# Delete rows that are in the specified groups
df1 = df1[~df1['category'].isin(groups_to_delete)]

In [405]:
def rename_category(row):
    conditions_mozarella = (
        "MOZARELLA" in row['name'], 
        "MOZZARELLA" in row['name'], 
        "MAZARELLA" in row['name'], 
        "MAZƏRELLA" in row['name'], 
        "MAZAERELLA" in row['name'], 
        "MAZARELLA" in row['name']
    )
    
    conditions_gouda = (
        "GAUDA" in row['name'], 
        "GOUDA" in row['name'], 
        "QOUDA" in row['name'], 
        "QAUDA" in row['name']
    )
    
    if any(conditions_mozarella):
        return "MOZARELLA"
    elif any(conditions_gouda):
        return "GOUDA"
    else:
        return row['category']

# Apply the function to the dataframe
df1['category'] = df1.apply(rename_category, axis=1)


In [406]:
import datetime as dt
df1['tarix'] = pd.to_datetime("today").strftime("%d/%m/%Y")
df1 = df1[['tarix', 'sku','brand','category','name','qramaj','yag_faiz','regular_price', 'sale_price','endirim_faiz','link']]
df1.head()

,tarix,sku,brand,category,name,qramaj,yag_faiz,regular_price,sale_price,endirim_faiz,link
0,31/07/2023,30131891,PINAR,ERIDILMIŞ PENDIR,PİNAR PEYNİR LABNE 100 GR,100.0,,2.80,2.39,14.64,https://bazarstore.az/en/products/pi%CC%87nar-peyni%CC%87r-labne-100-gr?_pos=1&_fid=0445217ef&_ss=c
1,31/07/2023,30012657,PRESIDENT,ÖZƏL PENDIRLƏR,PREZİDENT DİLİMLİ MAAZDAM PENDİR 150 QR,150.0,,4.10,3.65,10.98,https://bazarstore.az/en/products/prezi%CC%87dent-di%CC%87li%CC%87mli%CC%87-maazdam-pendi%CC%87r-150-qr?_pos=2&_fid=0445217ef&_ss=c
2,31/07/2023,30012634,HOCHLAND,ERIDILMIŞ PENDIR,HOCHLAND DİLİMLİ PENDİR ÇİZBURQER 150 QR,150.0,,3.99,3.35,16.04,https://bazarstore.az/en/products/hochland-di%CC%87li%CC%87mli%CC%87-pendi%CC%87r-ci%CC%87zburqer-150-qr?_pos=3&_fid=0445217ef&_ss=c
3,31/07/2023,30122302-1000,OLDENBUGER,GOUDA,OLDENBURGER GOUDA 48% YAGLI PENDIR 1000 KG,1000.0,48.0,18.70,NaN,NaN,https://bazarstore.az/en/products/oldenburger-gouda-48-yagli-pendir-kg?_pos=4&_fid=0445217ef&_ss=c
4,31/07/2023,30101690,MILK PRO,AĞ PENDIR ƏDƏD,GÖYÇAY SÜD ÇANAX PENDİRİ 500 GR,500.0,,5.49,NaN,NaN,https://bazarstore.az/en/products/goycay-sud-canax-pendi%CC%87ri%CC%87-500-gr?_pos=5&_fid=0445217ef&_ss=c


In [407]:
TodaysDate = time.strftime("%d-%m-%Y")
excelfilename = '/Users/FaiqRasulov/Desktop/Qiymet_list/bazarstore_qiymet_list ' + TodaysDate +".xlsx"

df1.to_excel(excelfilename, sheet_name='BazarStore '+ TodaysDate, index=False, header=True)